In [1]:
import pandas as pd
import os
import dask.dataframe as dd
from pathlib import Path
import subprocess as sp
import math
import seaborn as sns
import matplotlib.pyplot as plt
import re
import numpy as np
import itertools

In [2]:
def genome_TSS_annotation(genome_path_gtf, TSS_gtf):
    """
    Takes a ensemble genome gtf file as input, and returns a smaller gtf file containing only the genes and
    their TSS locations
    """
    Path(f"{TSS_gtf}").touch()
    tempfile = os.getcwd() + "/temp_TSS_genome_annotation.txt"
    Path(tempfile).touch()
    os.remove(f"{TSS_gtf}")
    # i = 0
    with open(f"{tempfile}", "a") as output_gtf:
        with open(genome_path_gtf) as f:
            for _ in range(5):
                next(f)
            for line in f:
                line = line.strip()
                line = line.split()
                gene_name = line[13]
                coding_stuff = line[17]
                strand_info = line[6]
                #             if i == 1000:
                #                 break
                #             i = i + 1
                gene_name = gene_name.rstrip(";")
                coding_stuff = coding_stuff.rstrip(";")
                # coding_stuff = coding_stuff.rstrip(";")
                if str(coding_stuff) == '"protein_coding"':
                    if line[6] == "+":
                        start = line[3]
                        end = int(start) + 1
                        output_gtf.write(
                            line[0]
                            + "\t"
                            + str(start)
                            + "\t"
                            + str(end)
                            + "\t"
                            + gene_name
                            + "\n"
                        )
                    else:
                        start = int(line[4]) - 1
                        end = line[4]
                        output_gtf.write(
                            line[0]
                            + "\t"
                            + str(start)
                            + "\t"
                            + str(end)
                            + "\t"
                            + gene_name
                            + "\n"
                        )
    sp.check_call(
        f"nice -5 bedtools sort " f"-i {tempfile} " f"> {TSS_gtf}", shell=True,
    )
    os.remove(tempfile)

In [3]:
def TSS_window_to_region(genome_TSS_gtf, bedregion_file, TSS_window):
    """
    Takes a genome TSS gtf file or bedfile as input (see the 'genome_TSS_GTF' script) together with a bed file 
    with regions and a window length.
    It wil subsequently take a window around the TSS, and list all regions overlapping with this window. It will
    furhtermore calculate the distance of te bedregion to the TSS.
    It will return a dataframe containing all these results.
    """
    sp.check_call(
        f"nice -10 bedtools window "
        f"-w {TSS_window} "
        f"-a {genome_TSS_gtf} "
        f"-b {bedregion_file} "
        f"> tmp_bedtool_window.txt",
        shell=True,
    )
    pd_table = pd.read_table(
        "tmp_bedtool_window.txt",
        index_col=None,
        names=[
            "chr_TSS",
            "start_TSS",
            "end_TSS",
            "gene_name",
            "Chrom",
            "ChromStart",
            "ChromEnd",
        ],
        dtype={
            "chr_TSS": str,
            "start_TSS": int,
            "end_TSS": int,
            "gene_name": str,
            "Chrom": str,
            "ChromStart": int,
            "ChromEnd": int,
        },
    )
    os.remove("tmp_bedtool_window.txt")
    distance = (
        []
    )  # calculate the distance based on the TSS start end region start site, check which one is upstream for distance calc.
    for x, y in zip(pd_table["start_TSS"], pd_table["ChromStart"]):
        if x > y:
            distance.append(x - y)
        elif y > x:
            distance.append(y - x)
        else:
            distance.append("NaN")
    pd_table["distance"] = distance
    return pd_table

In [4]:
def distance_weight(include_promoter=False, include_enhancer=True, alpha=1e5, padding=100000, keep1=5000, remove=2000):
    """
    Built weight distribution from TSS. (From Ananse)
    """
    # alpha is half site, default setting is 1e5, which means at 1e5 position weight is 0.5
    # padding is the full range we used
    # remove is promoter removed range
    # keep1 is keep full binding score range
    u = -math.log(1.0 / 3.0) * 1e5 / alpha
    if include_promoter and include_enhancer:
        weight1 = pd.DataFrame(
            {"weight": [1 for z in range(1, remove + 1)], "dist": range(1, remove + 1)}
        )
        weight2 = pd.DataFrame(
            {
                "weight": [1 for z in range(remove + 1, keep1 + 1)],
                "dist": range(remove + 1, keep1 + 1),
            }
        )
        weight3 = pd.DataFrame(
            {
                "weight": [
                    2.0
                    * math.exp(-u * math.fabs(z) / 1e5)
                    / (1.0 + math.exp(-u * math.fabs(z) / 1e5))
                    for z in range(1, padding - keep1 + 1)
                ],
                "dist": range(keep1 + 1, padding + 1),
            }
        )
    elif not include_promoter and include_enhancer:
        weight1 = pd.DataFrame(
            {"weight": [0 for z in range(1, remove + 1)], "dist": range(1, remove + 1)}
        )
        weight2 = pd.DataFrame(
            {
                "weight": [1 for z in range(remove + 1, keep1 + 1)],
                "dist": range(remove + 1, keep1 + 1),
            }
        )
        weight3 = pd.DataFrame(
            {
                "weight": [
                    2.0
                    * math.exp(-u * math.fabs(z) / 1e5)
                    / (1.0 + math.exp(-u * math.fabs(z) / 1e5))
                    for z in range(1, padding - keep1 + 1)
                ],
                "dist": range(keep1 + 1, padding + 1),
            }
        )
    elif include_promoter and not include_enhancer:
        weight1 = pd.DataFrame(
            {"weight": [1 for z in range(1, remove + 1)], "dist": range(1, remove + 1)}
        )
        weight2 = pd.DataFrame(
            {
                "weight": [0 for z in range(remove + 1, keep1 + 1)],
                "dist": range(remove + 1, keep1 + 1),
            }
        )
        weight3 = pd.DataFrame(
            {
                "weight": [0 for z in range(1, padding - keep1 + 1)],
                "dist": range(keep1 + 1, padding + 1),
            }
        )
    else:
        weight1 = pd.DataFrame(
            {"weight": [0 for z in range(1, remove + 1)], "dist": range(1, remove + 1)}
        )
        weight2 = pd.DataFrame(
            {
                "weight": [0 for z in range(remove + 1, keep1 + 1)],
                "dist": range(remove + 1, keep1 + 1),
            }
        )
        weight3 = pd.DataFrame(
            {
                "weight": [0 for z in range(1, padding - keep1 + 1)],
                "dist": range(keep1 + 1, padding + 1),
            }
        )
    weight = pd.concat([weight1, weight2, weight3])
    weight_dict = pd.Series(weight.weight.values, index=weight.dist).to_dict()
    return weight_dict

In [5]:
#load the table to convert the chromosome names to the USCS format
CRCH38_ensembl2UCSC = pd.read_csv('/scratch/bacint/jdeleuw/KC_ATAC/analysis/bigBed/CRCH38_ensembl2UCSC.txt', sep ='\t',header = 0, index_col=False)

#make dictionary which can be used to convert ensembl to UCSC format 
ensembl_to_UCSC = {}
UCSC_to_ensembl = {}
for row in CRCH38_ensembl2UCSC.itertuples():
    ensembl_to_UCSC[row[1]] = row[2]
    UCSC_to_ensembl[row[2]] = row[1]

# Load data

In [6]:
#import TFs of interest
TFs = pd.read_csv('/scratch/bacint/jdeleuw/ananse/analysis/interesting_TFs.csv', header = None)

In [7]:
variable_names = pd.read_csv('/scratch/bacint/jdeleuw/dev_ananse/binding_data/binding_analysis/Variable_names.csv', sep = ',', header = 0, index_col = 0)

In [8]:
#import binding files
os.chdir('/scratch/bacint/jdeleuw/dev_ananse/binding_data')
for index in variable_names.index:
    path_list = [os.getcwd(), '/binding_', index, '/binding.tsv']
    path = ''.join(path_list)
    globals()[index] = dd.read_csv(path, sep='\t', engine='python')

# Filter TFs

In [ ]:
#filter TFs of interest from binding files
binding_dfs = [globals()[i] for i in dir() if re.match('^day.', i)]
binding_dfs.extend([globals()[i] for i in dir() if re.match('KC', i)])

for name, binding_df in zip(variable_names.TF_filtered_binding_dfs, binding_dfs):
    globals()[name] = binding_df[binding_df['factor'].isin(TFs[0].tolist())].compute()

# Get all the enhancer within 100 kb window of TSS and calculate the distance

In [17]:
TF_filtered_dfs = [globals()[i] for i in dir() if re.search('^filtered_Day.$', i) 
                   or re.search('^filtered_Day..$', i) 
                   or re.search('^filtered_KC$', i)]

for df, day in zip(TF_filtered_dfs, variable_names.index):
    enhancer_duplicates = df['enhancer'].str.split(':|-', n = 2, expand = True)
    enhancer = enhancer_duplicates.drop_duplicates()
    enhancer_ensembl = enhancer.replace({0 : UCSC_to_ensembl})
    
    path_list = ['/scratch/bacint/jdeleuw/dev_ananse/binding_data/binding_analysis/bed_files/', day, '.bed']
    path = ''.join(path_list)
    
    enhancer_ensembl.to_csv(path, sep = '\t', header = False, index = False)

In [18]:
#subset the TSS of each gene from the GTF file
genome_TSS_annotation('/scratch/bacint/jdeleuw/genomes/GRCh38.p13/GRCh38.p13.annotation.gtf', 
                      '/scratch/bacint/jdeleuw/genomes/GRCh38.p13/GRCh38.p13.small.annotation.gtf')

In [19]:
#Make df with TSS and enhancers within 100 kb
TF_filtered_dfs = [globals()[i] for i in dir() if re.search('^filtered_Day.$', i) 
                   or re.search('^filtered_Day..$', i) 
                   or re.search('^filtered_KC$', i)]

for binding_df, day, output_name in zip(TF_filtered_dfs, variable_names.index, variable_names.distance_TSS_enhancer):
    path_list = ['/scratch/bacint/jdeleuw/dev_ananse/binding_data/binding_analysis/bed_files/', day, '.bed']
    path = ''.join(path_list)
    
    df = TSS_window_to_region('/scratch/bacint/jdeleuw/genomes/GRCh38.p13/GRCh38.p13.small.annotation.gtf',
                                                   path,
                                                   100000)
    #filter TF of interest(influence_score >= 0.95) from the df
    filtered_df = df[df['gene_name'].isin(TFs[0].tolist())]
    
    #merge df with TSS and linked enhancers, with the factor binding to the enhancer with the binding score
    filtered_df = filtered_df.replace({'Chrom': ensembl_to_UCSC})
    filtered_df['enhancer'] = filtered_df[['Chrom', 'ChromStart']].astype(str).agg(':'.join, axis=1)
    filtered_df['enhancer'] = filtered_df[['enhancer', 'ChromEnd']].astype(str).agg('-'.join, axis=1)
    
    if day == 'day0':
        merged_df_temp = filtered_df.merge(binding_df, how = 'inner', on = 'enhancer')
        merged_df = merged_df_temp.rename(columns = {'binding' : 'binding_day0'})
    else: 
        merged_df['binding_' + day] = filtered_df.merge(binding_df, how = 'inner', on = 'enhancer')['binding']

# Normalize the binding score based on the distance
# How are the TFs of each cluster targeted by all the TFs

In [20]:
Cluster1 = ['NANOG', 'SOX2', 'SALL4', 'PRDM14', 'ZSCAN10', 'OTX2', 'ZIC3', 'ZFP42', 'MYCN', 'FOXH1']
Cluster2 = ['MYC']
Cluster3 = ['MEIS1', 'HAND1', 'MSX2', 'HOXB2', 'MEIS2', 'HEY1', 'TBX3', 'TFAP2B']
Cluster4 = ['TP63', 'MAF', 'EHF', 'HES2']
Cluster5 = ['EPAS1', 'CEBPD', 'KLF5', 'IRF6', 'GRHL1', 'FOSL2', 'SNAI2']
Cluster6 = ['IRF9', 'IRX3', 'OSR1', 'HOXB1']

In [21]:
#add weight based on the distance from the binding site to the TSS
Cluster_list = [globals()[i] for i in dir() if re.search('^Cluster.$', i)]

weights = distance_weight(alpha=1e4, include_promoter=False)
test = merged_df.copy()

for Cluster, output_name in zip(Cluster_list, variable_names.targeted_cluster_hm):
    hm_data = pd.DataFrame(index = TFs[0], columns = variable_names.index)
    for day in variable_names.index:
        test['weight'] = test['distance'].map(weights)
        test['binding_weight_' + day] = test['binding_' + day] * test['weight']      

        regulation_probability = pd.DataFrame(index = TFs[0], columns = TFs[0])
        for target_gene, TF in itertools.product(Cluster, TFs[0]):
            filtered_target = test[test['gene_name'].isin([target_gene])]
            filtered_TF = filtered_target[filtered_target['factor'].isin([TF])]

            regulation_probability.at[target_gene, TF] = filtered_TF['binding_weight_' + day].sum()
        for TF in TFs[0]:
            regulation_probability.replace(0, np.nan, inplace=True)
            hm_data.at[TF, day] = regulation_probability.loc[:, TF].mean()
        globals()[output_name] = hm_data
        path_list = ['/scratch/bacint/jdeleuw/dev_ananse/binding_data/binding_analysis/', output_name, '.tsv']
        path = ''.join(path_list)
        hm_data.to_csv(path, sep='\t')

# from the old run everything before this point is identical

In [21]:
#Make df with TSS and enhancers within 100 kb
TF_filtered_dfs = [globals()[i] for i in dir() if re.search('^filtered_Day.$', i) 
                   or re.search('^filtered_Day..$', i) 
                   or re.search('^filtered_KC$', i)]

for binding_df, day, output_name in zip(TF_filtered_dfs, variable_names.index, variable_names.distance_TSS_enhancer):
    path_list = ['/scratch/bacint/jdeleuw/dev_ananse/binding_data/binding_analysis/bed_files/', day, '.bed']
    path = ''.join(path_list)
    
    df = TSS_window_to_region('/scratch/bacint/jdeleuw/genomes/GRCh38.p13/GRCh38.p13.small.annotation.gtf',
                                                   path,
                                                   100000)
    #filter TF of interest(influence_score > 0.95) from te df
    filtered_df = df[df['gene_name'].isin(TFs[0].tolist())]
    
    #merge df with TSS and linked enhancers, with the factor binding to the enhancer with the binding score
    filtered_df = filtered_df.replace({'Chrom': ensembl_to_UCSC})
    filtered_df['enhancer'] = filtered_df[['Chrom', 'ChromStart']].astype(str).agg(':'.join, axis=1)
    filtered_df['enhancer'] = filtered_df[['enhancer', 'ChromEnd']].astype(str).agg('-'.join, axis=1)
    
    globals()[output_name] =  filtered_df.merge(binding_df, how = 'inner', on = 'enhancer')

In [22]:
# test = distance_Day0[distance_Day0['gene_name'].isin(['HES2'])]
# test['enhancer'].unique()
distance_Day0

,chr_TSS,start_TSS,end_TSS,gene_name,Chrom,ChromStart,ChromEnd,distance,enhancer,factor,binding
0,1,6424669,6424670,HES2,chr1,6325544,6325745,99125,chr1:6325544-6325745,EPAS1,0.34548
1,1,6424669,6424670,HES2,chr1,6325544,6325745,99125,chr1:6325544-6325745,SNAI2,0.18312
2,1,6424669,6424670,HES2,chr1,6325544,6325745,99125,chr1:6325544-6325745,MSX2,0.20395
3,1,6424669,6424670,HES2,chr1,6325544,6325745,99125,chr1:6325544-6325745,HOXB2,0.31641
4,1,6424669,6424670,HES2,chr1,6325544,6325745,99125,chr1:6325544-6325745,NANOG,0.11331
...,...,...,...,...,...,...,...,...,...,...,...
29881,8,144475848,144475849,FOXH1,chr8,144575536,144575737,99688,chr8:144575536-144575737,GRHL1,0.18814
29882,8,144475848,144475849,FOXH1,chr8,144575536,144575737,99688,chr8:144575536-144575737,IRF9,0.43239
29883,8,144475848,144475849,FOXH1,chr8,144575536,144575737,99688,chr8:144575536-144575737,TBX3,0.45133
29884,8,144475848,144475849,FOXH1,chr8,144575536,144575737,99688,chr8:144575536-144575737,CEBPD,0.31083


In [23]:
Cluster1 = ['NANOG', 'SOX2', 'SALL4', 'PRDM14', 'ZSCAN10', 'OTX2', 'ZIC3', 'ZFP42', 'MYCN', 'FOXH1']
Cluster2 = ['MYC']
Cluster3 = ['MEIS1', 'HAND1', 'MSX2', 'HOXB2', 'MEIS2', 'HEY1', 'TBX3', 'TFAP2B']
Cluster4 = ['TP63', 'MAF', 'EHF', 'HES2']
Cluster5 = ['EPAS1', 'CEBPD', 'KLF5', 'IRF6', 'GRHL1', 'FOSL2', 'SNAI2']
Cluster6 = ['IRF9', 'IRX3', 'OSR1', 'HOXB1']

In [42]:
#add weight based on the distance from the binding site to the TSS
distance_dfs = [globals()[i] for i in dir() if re.search('^distance_Day.$', i)
                or re.search('^distance_Day..$', i) 
                or re.search('^distance_KC$', i)]
Cluster_list = [globals()[i] for i in dir() if re.search('^Cluster.$', i)]

weights = distance_weight(alpha=1e4, include_promoter=False)

for Cluster, output_name in zip(Cluster_list, variable_names.targeted_cluster_hm):
    hm_data = pd.DataFrame(index = TFs[0], columns = variable_names.index)
    for df, day in zip(distance_dfs, variable_names.index):
        df['weight'] = df['distance'].map(weights)
        df['binding_weight'] = df['binding'] * df['weight']      

        regulation_probability = pd.DataFrame(index = TFs[0], columns = TFs[0])
        for target_gene, TF in itertools.product(TFs[0], TFs[0]):
            filtered_target = df[df['gene_name'].isin([target_gene])]
            filtered_TF = filtered_target[filtered_target['factor'].isin([TF])]

            regulation_probability.at[target_gene, TF] = filtered_TF['binding_weight'].sum()
        for TF in TFs[0]:
            regulation_probability.replace(0, np.nan, inplace=True)
            hm_data.at[TF, day] = regulation_probability.loc[:, TF].median()
    globals()[output_name] = hm_data
    path_list = ['/scratch/bacint/jdeleuw/dev_ananse/binding_data/binding_analysis/', output_name, '2.tsv']
    path = ''.join(path_list)
    hm_data.to_csv(path, sep='\t')
# hm_data_all.to_csv('/scratch/bacint/jdeleuw/dev_ananse/binding_data/binding_analysis/avg_binding.tsv', sep = '\t')

In [43]:
hm_data

Day,day0,day07,day14,day21,day43,day60,KC
0,,,,,,,
CEBPD,1.288474,1.408854,1.335435,1.522592,1.370805,1.236190,1.357187
EHF,1.288540,1.531236,1.723425,1.569336,1.467538,1.417249,1.386874
EPAS1,1.876633,1.967900,1.948129,1.898744,1.960539,1.820026,1.911307
FOSL2,1.678364,2.045805,1.810959,1.601668,1.772242,1.736251,1.664599
FOXH1,1.409630,1.720509,1.767127,1.620360,1.550774,1.486554,1.526097
GRHL1,1.542192,1.790372,1.549551,1.741715,1.663205,1.582146,1.502585
HAND1,1.510161,1.570748,1.577544,1.676905,1.658619,1.462567,1.494010
HES2,2.005505,2.021125,2.021099,2.051447,2.011806,2.004647,2.009485
HEY1,1.429732,1.545142,1.676679,1.651011,1.465219,1.508449,1.447178


In [45]:
#add weight based on the distance from the binding site to the TSS
distance_dfs = [globals()[i] for i in dir() if re.search('^distance_Day.$', i)
                or re.search('^distance_Day..$', i) 
                or re.search('^distance_KC$', i)]
Cluster_list = [globals()[i] for i in dir() if re.search('^Cluster.$', i)]

weights = distance_weight(alpha=1e4, include_promoter=False)

for Cluster, output_name in zip(Cluster_list, variable_names.targeting_cluster_hm):
    hm_data = pd.DataFrame(index = TFs[0], columns = variable_names.index)
    for df, day in zip(distance_dfs, variable_names.index):
        df['weight'] = df['distance'].map(weights)
        df['binding_weight'] = df['binding'] * df['weight']      
        
        regulation_probability = pd.DataFrame(index = TFs[0], columns = TFs[0])
        for target_gene, TF in itertools.product(Cluster, TFs[0]):
            filtered_target = df[df['gene_name'].isin([target_gene])]
            filtered_TF = filtered_target[filtered_target['factor'].isin([TF])]

            regulation_probability.at[target_gene, TF] = filtered_TF['binding_weight'].sum()
        for TF in TFs[0]:
            regulation_probability.replace(0, np.nan, inplace=True)
            hm_data.at[TF, day] = regulation_probability.loc[:, TF].mean()
    globals()[output_name] = hm_data
    path_list = ['/scratch/bacint/jdeleuw/dev_ananse/binding_data/binding_analysis/', output_name, '2.tsv']
    path = ''.join(path_list)
    hm_data.to_csv(path, sep='\t')

In [46]:
hm_data

Day,day0,day07,day14,day21,day43,day60,KC
0,,,,,,,
CEBPD,1.789992,2.356418,1.934651,1.714610,1.519043,1.538332,1.455296
EHF,1.892087,2.556373,2.110531,1.840660,1.544962,1.573766,1.518893
EPAS1,2.098745,2.592621,2.252064,2.050217,1.860546,1.897103,1.826770
FOSL2,1.690914,2.032707,1.759675,1.596311,1.501649,1.550880,1.468265
FOXH1,1.749085,2.259563,1.859677,1.666911,1.526133,1.560329,1.468322
GRHL1,1.900212,2.424015,2.048217,1.819320,1.611624,1.681833,1.584648
HAND1,1.765074,2.252119,1.915584,1.689119,1.522303,1.580482,1.496332
HES2,1.940485,2.180157,2.024199,1.925941,1.842738,1.862760,1.803689
HEY1,1.795266,2.319263,1.932877,1.751036,1.582067,1.618294,1.529095


In [15]:
sns.set(rc={"figure.figsize": (15, 3)})
# import from Ananse
weight = distance_weight(alpha=1e4, include_promoter=False)
sns.lineplot(x=weight["dist"], y=weight["weight"])
plt.savefig(f"/scratch/bacint/jdeleuw/dev_ananse/binding_data/binding_analysis/weight_distance_enhancer_window.pdf")
enh_weight_dict = pd.Series(weight.weight.values, index=weight.dist).to_dict()

KeyError: 'dist'

In [163]:
#add weight based on the distance from the binding site to the TSS normalize it by substracting row means
distance_dfs = [globals()[i] for i in dir() if re.search('^distance_Day.$', i)
                or re.search('^distance_Day..$', i) 
                or re.search('^distance_KC$', i)]
Cluster_list = [globals()[i] for i in dir() if re.search('^Cluster.$', i)]

weights = distance_weight(alpha=1e4, include_promoter=False)

# for Cluster, output_name in zip(Cluster_list, variable_names.targeted_cluster_hm):
hm_data_row_mean = pd.DataFrame(index = TFs[0], columns = variable_names.index)
for df, day in zip(distance_dfs, variable_names.index):
    df['weight'] = df['distance'].map(weights)
    df['binding_weight'] = df['binding'] * df['weight']      

    regulation_probability = pd.DataFrame(index = TFs[0], columns = TFs[0])
    for target_gene, TF in itertools.product(TFs[0], TFs[0]):
        filtered_target = df[df['gene_name'].isin([target_gene])]
        filtered_TF = filtered_target[filtered_target['factor'].isin([TF])]

        regulation_probability.at[target_gene, TF] = filtered_TF['binding_weight'].sum()
    for TF in TFs[0]:
        regulation_probability.replace(0, np.nan, inplace=True)
        #substract row_means from the values of each row
        row_means = regulation_probability.mean(axis=1)
        norm_regulation_probability = regulation_probability.loc[TF, :] - regulation_probability.mean(axis=1)[TF]
                
        hm_data_row_mean.at[TF, day] = norm_regulation_probability.mean()
hm_data_row_mean.to_csv('/scratch/bacint/jdeleuw/dev_ananse/binding_data/binding_analysis/avg_binding_row_mean.tsv', sep = '\t')